# 光学式文字認識

![新聞を読んでいるロボット](./images/ocr.jpg)

画像の中からテキストを検出して解釈することは、Computer Vision における一般的なタスクです。この種の処理は、通常、*光学式文字認識* (OCR) と呼ばれます。

## Computer Vision サービスを使用して画像の中にあるテキストを見つける

**Computer Vision** Cognitive Service は、次のような OCR タスクをサポートしています。

- 複数の言語のテキストを読み取れる **OCR** API。この API は同期的に使用でき、画像の中から少量のテキストを検出して読み取る必要がある場合にうまく機能します。
- サイズの大きいドキュメント用に最適化された **Read** API。この API は非同期で使用し、印刷されたテキストと手書きのテキストの両方を読み取れます。

このサービスを使用するには、**Computer Vision** リソースまたは **Cognitive Services** リソースのいずれかを作成します。

まだ作成していない場合は、Azure サブスクリプションに **Cognitive Services** リソースを作成します。

> **注**: Cognitive Services リソースが既にある場合は、Azure portal で**クイック スタート**ページを開き、キーとエンドポイントを以下のセルにコピーするだけで作成できます。それ以外の場合は、以下の手順に従って作成してください。

1. ブラウザーの新しいタブで Azure portal (https://portal.azure.com) を開き、Microsoft アカウントでサインインします。

2. 「**&#65291;リソースの作成**」 ボタンをクリックし、*Cognitive Services* を検索して、以下の設定で **Cognitive Services** リソースを作成します。
    - **サブスクリプション**: *使用する Azure サブスクリプション*
    - **リソース グループ**: *一意の名前のリソース グループを選択または作成します*
    - **リージョン**: *利用可能な任意のリージョンを選択します*。
    - **名前**: *一意の名前を入力します*。
    - **価格レベル**: S0
    - **注意事項を読み理解しました**: 選択されています。
3. デプロイが完了するまで待ちます。そのあと Cognitive Services リソースに移動し、「**概要**」 ページでリンクをクリックしてサービスのキーを管理します。クライアント アプリケーションから Cognitive Services リソースに接続するには、エンドポイントとキーが必要です。

### Cognitive Services リソースのキーとエンドポイントを取得する

Cognitive Services リソースを使用するには、クライアント アプリケーションにエンドポイントと認証キーが必要です。

1. Azure portalで、Cognitive Services リソースの 「**キーとエンドポイント**」 ページからリソースの 「**キー 1**」 の値をコピーし、以下のコードに貼り付けます (**YOUR_COG_KEY** と置き換える)。
2. リソースの**エンドポイント**をコピーして以下のコードに貼り付けます (**YOUR_COG_ENDPOINT** と置き換える)。
3. セルの左側にある 「**セルの実行**」  (&#9655;) ボタンをクリックして、以下のセルのコードを実行します。

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

キーとエンドポイントを設定したので、Computer Vision サービスのリソースを使用して画像からテキストを抽出できるようになりました。

**OCR** API から始めます。これにより、画像を同期的に分析し、画像に含まれるテキストを読み取ることができます。この例では、架空の小売店 Northwind Traders の広告画像にテキストが含まれています。以下のセルを実行してテキストを読み取ります。 

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import os
%matplotlib inline

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Read the image file
image_path = os.path.join('data', 'ocr', 'advert.jpg')
image_stream = open(image_path, "rb")

# Use the Computer Vision service to find text in the image
read_results = computervision_client.recognize_printed_text_in_stream(image_stream)

# Process the text line by line
for region in read_results.regions:
    for line in region.lines:

        # Read the words in the line of text
        line_text = ''
        for word in line.words:
            line_text += word.text + ' '
        print(line_text.rstrip())

# Open image to display it.
fig = plt.figure(figsize=(7, 7))
img = Image.open(image_path)
draw = ImageDraw.Draw(img)
plt.axis('off')
plt.imshow(img)

画像の中から見つかったテキストは範囲、行、単語の階層構造で整理され、コードはこれらを読み取って結果を取得します。

その結果、画像の中から読み取ったテキストを表示します。 

## 境界ボックスを表示する

結果には、画像の中にあるテキスト行と個々の単語の*境界ボックス*座標も含まれます。以下のセルを実行して、上記で取得した広告画像のテキスト行の境界ボックスを確認します。

In [ ]:
# Open image to display it.
fig = plt.figure(figsize=(7, 7))
img = Image.open(image_path)
draw = ImageDraw.Draw(img)

# Process the text line by line
for region in read_results.regions:
    for line in region.lines:

        # Show the position of the line of text
        l,t,w,h = list(map(int, line.bounding_box.split(',')))
        draw.rectangle(((l,t), (l+w, t+h)), outline='magenta', width=5)

        # Read the words in the line of text
        line_text = ''
        for word in line.words:
            line_text += word.text + ' '
        print(line_text.rstrip())

# Show the image with the text locations highlighted
plt.axis('off')
plt.imshow(img)

その結果、テキストの各行の境界ボックスが画像上に長方形として表示されます。

## Read API を使用する

前に使用した OCR API は、テキストが少ない画像に適しています。スキャンした文書など、より多くのテキストを読み取る必要がある場合は、**Read** API を使用します。次の各手順に従います。

1. 画像を Computer Vision サービスに送信して、非同期で読み取って分析します。
2. 分析処理が完了するまで待ちます。
3. 分析の結果を取得します。

次のセルを実行してこのプロセスを実施し、小売店 Northwind Traders のマネージャー宛ての手紙をスキャンした画像からテキストを読み取ります。

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image
import time
import os
%matplotlib inline

# Read the image file
image_path = os.path.join('data', 'ocr', 'letter.jpg')
image_stream = open(image_path, "rb")

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Submit a request to read printed text in the image and get the operation ID
read_operation = computervision_client.read_in_stream(image_stream,
                                                      raw=True)
operation_location = read_operation.headers["Operation-Location"]
operation_id = operation_location.split("/")[-1]

# Wait for the asynchronous operation to complete
while True:
    read_results = computervision_client.get_read_result(operation_id)
    if read_results.status not in [OperationStatusCodes.running]:
        break
    time.sleep(1)

# If the operation was successfuly, process the text line by line
if read_results.status == OperationStatusCodes.succeeded:
    for result in read_results.analyze_result.read_results:
        for line in result.lines:
            print(line.text)

# Open image and display it.
print('\n')
fig = plt.figure(figsize=(12,12))
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

結果を確認します。手紙の大部分は印刷されたテキストで、手書きの署名もありますが、完全に読み取られています。手紙の元の画像は OCR の結果の下に表示されています。表示するにはスクロールする必要があるかもしれません。

## 手書きのテキストを読み取る

前の例の画像分析リクエストでは、*印刷された*テキストの処理を最適化する、テキスト認識モードを指定していました。それにもかかわらず、手書きの署名も読み取っています。

手書きのテキストを読み取れるこの機能は非常に便利です。たとえば、買い物リストをメモに書き、スマートフォンのアプリを使用してメモを読み取り、メモの中からテキストを書き起こしてみます。

以下のセルを実行して、手書きの買い物リストの読み取り処理の例を確認します。

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image
import time
import os
%matplotlib inline

# Read the image file
image_path = os.path.join('data', 'ocr', 'note.jpg')
image_stream = open(image_path, "rb")

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Submit a request to read printed text in the image and get the operation ID
read_operation = computervision_client.read_in_stream(image_stream,
                                                      raw=True)
operation_location = read_operation.headers["Operation-Location"]
operation_id = operation_location.split("/")[-1]

# Wait for the asynchronous operation to complete
while True:
    read_results = computervision_client.get_read_result(operation_id)
    if read_results.status not in [OperationStatusCodes.running]:
        break
    time.sleep(1)

# If the operation was successfuly, process the text line by line
if read_results.status == OperationStatusCodes.succeeded:
    for result in read_results.analyze_result.read_results:
        for line in result.lines:
            print(line.text)

# Open image and display it.
print('\n')
fig = plt.figure(figsize=(12,12))
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

## 詳細情報

OCR 向け Computer Vision Services の詳細については、「[Computer Vision ドキュメント](https://docs.microsoft.com/ja-jp/azure/cognitive-services/computer-vision/concept-recognizing-text)」を参照してください。